In [1]:
print("ok")

ok


In [2]:
!pip install faiss-cpu
import numpy as np
!pip install langchain sentence-transformers ctransformers

In [3]:
# Import necessary libraries
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain import PromptTemplate
from langchain.chains import RetrievalQA

In [4]:
!pip install pypdf


load and split documents 

In [5]:
def load_pdf(data_directory):
    loader = DirectoryLoader(data_directory, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [6]:
def split_text(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(documents)
    return text_chunks

In [7]:
# Load and split your PDF documents
extracted_data = load_pdf("data/")
text_chunks = split_text(extracted_data)
print("Number of text chunks:", len(text_chunks))

Number of text chunks: 4747


Generate embeddings

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings

def get_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings = get_embeddings()


C:\Users\HP\AppData\Local\Temp\ipykernel_38488\55377394.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\HP\anaconda3\envs\medsstudychatbot\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\HP\anaconda3\envs\medsstudychatbot\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` 

In [9]:
embeddings 

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

Create FAISS vector store

In [10]:
from langchain.vectorstores import FAISS

# Create FAISS vector store from text chunks
docsearch = FAISS.from_texts([chunk.page_content for chunk in text_chunks], embeddings)


In [11]:
# Save FAISS index
docsearch.save_local("faiss_index")

# Load FAISS index with `allow_dangerous_deserialization=True`
docsearch = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)


Set Up RetrievalQA with FAISS

In [12]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import CTransformers

# Define your prompt template
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""


In [13]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}



In [14]:
# Initialize your LLM (Ensure the model path is correct)
llm = CTransformers(
    model=r'C:\Users\HP\OneDrive\Documents\GitHub\medical-study-chatbot-using-llama2\models\llama-2-7b-chat.ggmlv3.q4_0.bin',  # Note the change here
    model_type="llama",
    config={
        'max_new_tokens': 512,
        'temperature': 0.8
    }
)

In [15]:
qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
        return_source_documents=True,
        chain_type_kwargs=chain_type_kwargs
    )   

In [16]:
try:
    while True:
        user_input = input("Input Prompt (type 'exit' to quit): ")
        if user_input.lower() == "exit":
            print("Exiting the chatbot. Goodbye!")
            break
        result = qa.invoke({"query": user_input})  
        print("Response:", result["result"])
        
except KeyboardInterrupt:
    print("\nChatbot interrupted. Exiting...")


Response: Intravenous (IV) injection.
Response: The user is asking about the type of injection used for the drug in question. Intravenous (IV) injection is the type of injection used when the drug is injected directly into a vein, typically through a needle or cannula. This method allows the drug to be delivered directly into the bloodstream, bypassing the digestive system and potentially reducing side effects.
Exiting the chatbot. Goodbye!


In [17]:
import gradio as gr

In [19]:
def chatbot(user_input):
    if user_input.lower() == "exit":
        return "Exiting the chatbot. Goodbye!"
    
    result = qa({"query": user_input})
    response = result["result"]
    

    return response

# Create the Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Medical Pharmacology Chat bot")
    user_input = gr.Textbox(lines=2, placeholder="Enter your query here...", label="User Input")
    output = gr.Textbox(lines=10, label="Response")
    
    submit_btn = gr.Button("Submit")
    submit_btn.click(fn=chatbot, inputs=user_input, outputs=output)

# Launch the Gradio app
demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


C:\Users\HP\AppData\Local\Temp\ipykernel_38488\3237070371.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = qa({"query": user_input})
Traceback (most recent call last):
  File "c:\Users\HP\anaconda3\envs\medsstudychatbot\lib\site-packages\gradio\queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "c:\Users\HP\anaconda3\envs\medsstudychatbot\lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "c:\Users\HP\anaconda3\envs\medsstudychatbot\lib\site-packages\gradio\blocks.py", line 1935, in process_api
    result = await self.call_function(
  File "c:\Users\HP\anaconda3\envs\medsstudychatbot\lib\site-packages\gradio\blocks.py", line 1520, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "c:\Users\HP\anaconda3\envs\medsstudych